<a href="https://colab.research.google.com/github/vickitarro/track1-medical-retrieval-bm25/blob/main/track1_bm25_clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets rank-bm25 nltk scikit-learn tqdm


In [ ]:
import nltk
nltk.download("punkt")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from datasets import load_dataset

pubmedqa = load_dataset("pubmed_qa", "pqa_labeled")
print(pubmedqa)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

pqa_labeled/train-00000-of-00001.parquet:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['pubid', 'question', 'context', 'long_answer', 'final_decision'],
        num_rows: 1000
    })
})


In [ ]:
len(pubmedqa["train"])


1000

In [ ]:
# Normalize PubMedQA into a clean format

def normalize_pubmedqa(dataset_split):
    records = []
    for item in dataset_split:
        context = " ".join(item["context"]["contexts"])
        records.append({
            "question": item["question"],
            "context": context,
            "source": "PubMedQA"
        })
    return records

pubmed_records = normalize_pubmedqa(pubmedqa["train"])

print("Total normalized records:", len(pubmed_records))
print(pubmed_records[0])


Total normalized records: 1000
{'question': 'Do mitochondria play a role in remodelling lace plant leaves during programmed cell death?', 'context': 'Programmed cell death (PCD) is the regulated death of cells within an organism. The lace plant (Aponogeton madagascariensis) produces perforations in its leaves through PCD. The leaves of the plant consist of a latticework of longitudinal and transverse veins enclosing areoles. PCD occurs in the cells at the center of these areoles and progresses outwards, stopping approximately five cells from the vasculature. The role of mitochondria during PCD has been recognized in animals; however, it has been less studied during PCD in plants. The following paper elucidates the role of mitochondrial dynamics during developmentally regulated PCD in vivo in A. madagascariensis. A single areole within a window stage leaf (PCD is occurring) was divided into three areas based on the progression of PCD; cells that will not undergo PCD (NPCD), cells in ear

In [ ]:
from nltk.tokenize import word_tokenize

def chunk_text(text, chunk_size=400, overlap=50):
    tokens = word_tokenize(text)
    chunks = []

    for i in range(0, len(tokens), chunk_size - overlap):
        chunk = tokens[i:i + chunk_size]
        chunks.append(" ".join(chunk))

    return chunks


In [ ]:
import nltk
nltk.download("punkt")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
print(type(pubmed_records))
print(len(pubmed_records))
print(pubmed_records[0].keys())


<class 'list'>
1000
dict_keys(['question', 'context', 'source'])


In [ ]:
print(pubmed_records[0])


{'question': 'Do mitochondria play a role in remodelling lace plant leaves during programmed cell death?', 'context': 'Programmed cell death (PCD) is the regulated death of cells within an organism. The lace plant (Aponogeton madagascariensis) produces perforations in its leaves through PCD. The leaves of the plant consist of a latticework of longitudinal and transverse veins enclosing areoles. PCD occurs in the cells at the center of these areoles and progresses outwards, stopping approximately five cells from the vasculature. The role of mitochondria during PCD has been recognized in animals; however, it has been less studied during PCD in plants. The following paper elucidates the role of mitochondrial dynamics during developmentally regulated PCD in vivo in A. madagascariensis. A single areole within a window stage leaf (PCD is occurring) was divided into three areas based on the progression of PCD; cells that will not undergo PCD (NPCD), cells in early stages of PCD (EPCD), and ce

In [ ]:
def chunk_text(text, chunk_size=400, overlap=50):
    words = text.split()
    chunks = []

    step = chunk_size - overlap
    for i in range(0, len(words), step):
        chunk = words[i:i + chunk_size]
        chunks.append(" ".join(chunk))

    return chunks

print("Safe chunk_text loaded")


Safe chunk_text loaded


In [ ]:
# Re-create pubmed_records if missing
from datasets import load_dataset

dataset = load_dataset("pubmed_qa", "pqa_labeled")

pubmed_records = []
for item in dataset["train"]:
    pubmed_records.append({
        "question": item["question"],
        "context": item["context"],
        "source": "PubMedQA"
    })

print("Records loaded:", len(pubmed_records))
print(pubmed_records[0].keys())


Records loaded: 1000
dict_keys(['question', 'context', 'source'])


In [ ]:
from nltk.tokenize import sent_tokenize

def chunk_text(text, chunk_size=400, overlap=50):
    words = text.split()
    chunks = []
    start = 0

    while start < len(words):
        end = start + chunk_size
        chunk = " ".join(words[start:end])
        chunks.append(chunk)
        start += chunk_size - overlap

    return chunks


In [ ]:
def chunk_text(text, chunk_size=400, overlap=50):
    # Handle dict inputs safely
    if isinstance(text, dict):
        text = " ".join(str(v) for v in text.values())

    # Ensure text is string
    text = str(text)

    words = text.split()
    chunks = []
    start = 0

    while start < len(words):
        end = start + chunk_size
        chunk = " ".join(words[start:end])
        chunks.append(chunk)
        start += chunk_size - overlap

    return chunks


In [ ]:
# Chunk PubMedQA contexts into overlapping chunks
chunked_docs = []

for record in pubmed_records:
    chunks = chunk_text(record["context"])
    for chunk in chunks:
        if chunk.strip():  # safety check
            chunked_docs.append({
                "text": chunk,
                "source": record["source"]
            })

print("Total chunks created:", len(chunked_docs))
print(chunked_docs[0]["text"][:300])


Total chunks created: 1026
['Programmed cell death (PCD) is the regulated death of cells within an organism. The lace plant (Aponogeton madagascariensis) produces perforations in its leaves through PCD. The leaves of the plant consist of a latticework of longitudinal and transverse veins enclosing areoles. PCD occurs in the c


In [ ]:
def retrieve_bm25(query, k=5):
    tokenized_query = query.split()
    scores = bm25.get_scores(tokenized_query)

    top_k_indices = sorted(
        range(len(scores)),
        key=lambda i: scores[i],
        reverse=True
    )[:k]

    results = []
    for idx in top_k_indices:
        results.append({
            "score": scores[idx],
            "text": chunked_docs[idx]["text"]
        })

    return results


# Test query
query = "What causes high blood pressure?"
results = retrieve_bm25(query, k=5)

for i, res in enumerate(results, 1):
    print(f"\nResult {i}")
    print(res["text"][:300])



Result 1
The objectives of this study were to investigate longitudinal predictors of fear of recurrence in survivors of head and neck cancer (HNC) using Leventhal's Common Sense Model (CSM) as a framework. The research questions were as follows: (a) to what extent do HNC patients report fear of cancer recurr

Result 2
longitudinal descriptive study. 2 large nursing homes in Turin, Italy. 418 dependent elderly (83 males, 335 females, mean age 83.7+/-8.5 y, range 55-102) living in the nursing homes. the prevalence of peripheral arterial disease (PAD) was evaluated using a Doppler Ultrasound measurement of AAI (Ankl

Result 3
Diabetes mellitus (DM) is undiagnosed in approximately half of the patients actually suffering from the disease. In addition, the prevalence of DM is more than twice as high as in patients with periodontitis when compared to periodontally healthy subjects. Thus, a high number of patients with period

Result 4
To examine patterns of knowledge and attitudes among adul

In [ ]:
import numpy as np

def is_relevant(retrieved_text, gold_text):
    # Handle dict inputs safely (PubMedQA contexts can be structured)
    if isinstance(retrieved_text, dict):
        retrieved_text = " ".join(str(v) for v in retrieved_text.values())
    if isinstance(gold_text, dict):
        gold_text = " ".join(str(v) for v in gold_text.values())

    retrieved_text = str(retrieved_text)
    gold_text = str(gold_text)

    retrieved = set(retrieved_text.lower().split())
    gold = set(gold_text.lower().split())

    overlap = len(retrieved.intersection(gold)) / max(1, len(gold))
    return overlap > 0.2

def evaluate_retrieval(pubmed_records, bm25, chunked_docs, k_values=[5, 10]):
    results = {}

    for k in k_values:
        recall_hits = 0
        reciprocal_ranks = []
        ndcgs = []

        for record in pubmed_records:
            query = record["question"]
            gold_text = record["context"]

            tokenized_query = query.lower().split()
            scores = bm25.get_scores(tokenized_query)

            ranked_indices = np.argsort(scores)[::-1][:k]
            retrieved_texts = [chunked_docs[i]["text"] for i in ranked_indices]

            # Recall
            found = False
            for rank, text in enumerate(retrieved_texts, start=1):
                if is_relevant(text, gold_text):
                    recall_hits += 1
                    reciprocal_ranks.append(1 / rank)
                    ndcgs.append(1 / np.log2(rank + 1))
                    found = True
                    break

            if not found:
                reciprocal_ranks.append(0)
                ndcgs.append(0)

        results[f"Recall@{k}"] = recall_hits / len(pubmed_records)
        results[f"MRR@{k}"] = np.mean(reciprocal_ranks)
        results[f"nDCG@{k}"] = np.mean(ndcgs)

    return results


In [ ]:
!pip install rank-bm25


In [ ]:
from rank_bm25 import BM25Okapi

# Build BM25 index
tokenized_corpus = [doc["text"].lower().split() for doc in chunked_docs]
bm25 = BM25Okapi(tokenized_corpus)

print("BM25 index built on", len(tokenized_corpus), "documents")


BM25 index built on 1026 documents


In [ ]:
# Run evaluation on PubMedQA
metrics = evaluate_retrieval(
    pubmed_records=pubmed_records,
    bm25=bm25,
    chunked_docs=chunked_docs,
    k_values=[5, 10]
)

for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")


Recall@5: 0.9880
MRR@5: 0.9628
nDCG@5: 0.9692
Recall@10: 0.9930
MRR@10: 0.9635
nDCG@10: 0.9709


In [ ]:
metrics


{'Recall@5': 0.988,
 'MRR@5': np.float64(0.96285),
 'nDCG@5': np.float64(0.9692103976746468),
 'Recall@10': 0.993,
 'MRR@10': np.float64(0.9635499999999999),
 'nDCG@10': np.float64(0.9708571488886066)}

In [ ]:
# Collect failure cases (gold not retrieved in top-10)
failure_cases = []

for record in pubmed_records[:200]:  # sample subset for speed
    query = record["question"]
    gold_text = record["context"]

    tokenized_query = query.lower().split()
    scores = bm25.get_scores(tokenized_query)
    ranked_indices = np.argsort(scores)[::-1][:10]
    retrieved_texts = [chunked_docs[i]["text"] for i in ranked_indices]

    found = False
    for text in retrieved_texts:
        if is_relevant(text, gold_text):
            found = True
            break

    if not found:
        failure_cases.append({
            "query": query,
            "gold": gold_text,
            "retrieved": retrieved_texts[:3]
        })

print("Failure cases found:", len(failure_cases))


Failure cases found: 2


In [ ]:
# View sample failure cases
for i, case in enumerate(failure_cases[:5]):
    print(f"\n--- Failure {i+1} ---")
    print("Query:", case["query"])
    print("Top retrieved snippet:", case["retrieved"][0][:300])
    print("Gold snippet:", str(case["gold"])[:300])




--- Failure 1 ---
Query: Double balloon enteroscopy: is it efficacious and safe in a community setting?
Top retrieved snippet: ['The aim of this study was to assess the efficacy of ureteroscopy for lower ureteric stones without the use of fluoroscopy.', 'Between June 2001 and January 2005, a total of 110 patients with a mean age of 33.5 years (range 12-65) suffering from of lower ureteral calculi (below the upper margin of 
Gold snippet: {'contexts': ['From March 2007 to January 2011, 88 DBE procedures were performed on 66 patients. Indications included evaluation anemia/gastrointestinal bleed, small bowel IBD and dilation of strictures. Video-capsule endoscopy (VCE) was used prior to DBE in 43 of the 66 patients prior to DBE evalua

--- Failure 2 ---
Query: "Would a man smell a rose then throw it away?
Top retrieved snippet: ['Unicompartmental replacement can be an alternative to tibial osteotomy in younger, active patients with unicompartmental knee disease. In unicompartmental repl